In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reviews4.csv
/kaggle/input/reviews1.csv
/kaggle/input/reviews3.csv


In [19]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
import keras.utils as ku
import string

In [20]:
curr_dir = '/kaggle/input/'
reviews = []
for file in os.listdir(curr_dir):
    review9_df = pd.read_csv(curr_dir + file)
    reviews.extend(list(review9_df.body.values))

len(reviews)

30

In [21]:
def clean(text):
    text = "".join(k for k in text if k not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii",'ignore')
    return text 

text = [clean(x) for x in reviews]

In [22]:
text

['phone is not working after update i need immediate replacement or need my money back without further delay',
 'great for all specs except for the battery life battery drains faster than expected without that totally worth it',
 'the most complete and honest review  this phone is outstanding mostly in a really good sense although somewhere we have to settle cons ranked  1 no stereo speakers  although single mono speaker is good 2 headphone audio output c type  35 mm  whatever is above average but should have been better with greater bass and more fuller ambient sound like lg g7 thinq etc ill update my review about the cirrus logic dac and oneplus c type bullets later 3 camera depends upon good lighting conditions too much crop in front camera video due to eis blur in nightscape especially if theres movement in frame wide angle camera should have been a must 4 fingerprint scanner is like 90 hit and 10 miss for 97 hit and 3 miss input same finger twice or thrice proper placement of fing

In [23]:
tokenizer = Tokenizer()

def get_tokens(corpus):
    tokenizer.fit_on_texts(text)
    total_no_words = len(tokenizer.word_index) + 1
    input = []
    for line in text:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram = token_list[:i+1]
            input.append(n_gram)
    return input, total_no_words

input, total_no_words = get_tokens(text)
input[:10]

[[9, 2],
 [9, 2, 14],
 [9, 2, 14, 214],
 [9, 2, 14, 214, 149],
 [9, 2, 14, 214, 149, 121],
 [9, 2, 14, 214, 149, 121, 7],
 [9, 2, 14, 214, 149, 121, 7, 101],
 [9, 2, 14, 214, 149, 121, 7, 101, 464],
 [9, 2, 14, 214, 149, 121, 7, 101, 464, 102],
 [9, 2, 14, 214, 149, 121, 7, 101, 464, 102, 71]]

In [24]:
def padded_sequences(input):
    max_len = max([len(x) for x in input])
    input = np.array(pad_sequences(input, maxlen=max_len, padding='pre'))
    
    predictors, label = input[:,:-1],input[:,-1]
    label = ku.to_categorical(label, num_classes=total_no_words)
    return predictors, label, max_len

predictors, label, max_sequence_len = padded_sequences(input)
print(len(predictors))
print(total_no_words)

3913
1151


In [25]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(Bidirectional(LSTM(units=256)))
    model.add(Dropout(0.4))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam') 
    return model


model = create_model(max_sequence_len, total_no_words)

In [26]:
model.fit(predictors, label, epochs=100,batch_size=200)

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
2400/3913 [=================>............] - ETA: 11s - loss: 6.8631

KeyboardInterrupt: 

In [10]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [17]:
print (generate_text("I got the product", 12, model, max_sequence_len))

I Got The Product One Plus 6T But Not Received All Other Accessories With It
